# Chat Testing

Here I am going to be figuring out all of the different aspects of how the chatbot will work.

In [18]:
from openai import OpenAI
import json

In [34]:
# Load Chat API Key
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path="./.env")

# Access the OPENAI_API_KEY variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

DATABASE_URL = os.getenv("POSTGRES_URL")
DATABASE_USER = os.getenv("POSTGRES_USER")
DATABASE_PASS = os.getenv("POSTGRES_PASSWORD")
DATABASE_DB = os.getenv("POSTGRES_DATABASE")
DATABASE_HOST = os.getenv("POSTGRES_HOST")


### Basic Load of OpenAI

In [6]:
client = OpenAI(api_key=OPENAI_API_KEY)

try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {
              "role": "user",
              "content": "Write a haiku about recursion in programming."
          }
      ]
  )

  print(completion.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

Code calls itself back,  
Layers of logic unfold,  
Endless depth of thought.


### Differentiate between Recipe, General, or Lifestyle

In [11]:
user_question = "How can I make my life easier with the diets I need to follow?"

user_diet = "celiac diet and low fodmap diet"

rules = """
1. You are the first step in a longer chatbot which has the goal of helping users
with specific dietary needs to answer general questions about the dietary needs, provide lifestyle advice 
in the context of said dietary need, or provide recipes specific to the user's needs.
2. Your job is only to differentiate the requests, so the next request to the chatbot will be able to have
a specific format.
For example, recipes will be returned in a different format from lifestyle advice, which will be
different from general queries.
3. Your job is to differentiate between general queries, lifestyle tips, and recipe requests.
4. If someone is asking for a recipe, you will respond with only 'recipe'.
5. If someone is asking general questions about their dietary needs, you will answer with "general"
6. If someone is asking about things that will be useful general knowledge living with their dietary need, you will answer "lifestyle"
7. Anything the user asks which does not pertain their diet, a diet, or the lifestyle associated with that diet, or claryfying questions, answer "not applicable"
"""




try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": f"{rules}"},
          {"role": "assistant", "content": f"{user_diet}"},
          {
              "role": "user",
              "content": f"{user_question}"
          }
      ]
  )

  print(completion.choices[0].message.content)
  query = completion.choices[0].message.content
except Exception as e:
    print(f"An error occurred: {e}")


lifestyle


### Recipe Responses

In [25]:
# Create the chat completion request
response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": "You give short recipes given certain dietary conditions."
        },
        {
            "role": "assistant",
            "content": "User has gluten-free preference"
        },
        {
            "role": "user",
            "content": "I would like a basic recipe with chicken and vegetables which takes less than 30 minutes."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recipe_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "Recipe": {
                        "description": "The name of the recipe",
                        "type": "string"
                    },
                    "Ingredients": {
                        "description": "list of ingredients with measurements if needed",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "Prep Time": {
                        "description": "time in minutes to prepare this recipe",
                        "type": "integer"
                    },
                    "Instructions": {
                        "description": "Instructions to create this recipe",
                        "type": "string"
                    }
                },
                "additionalProperties": False
            }
        }
    }
)

# Print the response
print(response.choices[0].message.content)
recipe = json.loads(response.choices[0].message.content)

{"Recipe":"Quick Chicken and Vegetable Stir-Fry","Ingredients":["1 lb chicken breast, sliced into thin strips","2 cups mixed vegetables (bell peppers, broccoli, carrots)","2 tablespoons olive oil","3 cloves garlic, minced","2 tablespoons soy sauce (gluten-free)","Salt and pepper to taste","1 teaspoon sesame oil (optional)"],"Prep Time":15,"Instructions":"1. Heat olive oil in a large skillet over medium-high heat.\n2. Add minced garlic and stir for about 30 seconds until fragrant.\n3. Toss in the sliced chicken and cook until no longer pink, around 5-7 minutes.\n4. Add mixed vegetables and stir-fry for an additional 5-7 minutes until vegetables are tender.\n5. Pour in gluten-free soy sauce, and add salt, pepper, and sesame oil (if using). Stir well for 2 minutes.\n6. Serve hot, and enjoy your quick meal!"}


In [26]:
recipe['Recipe']

'Quick Chicken and Vegetable Stir-Fry'

In [ ]:
import pg8000


In [ ]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()

# Fetch table names from Database
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
""")
tables = cursor.fetchall()
print("Tables in the database:", [table[0] for table in tables])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'users';
""")
columns = cursor.fetchall()
print("Columns in 'users' table:", [col[0] for col in columns])

# Example query: Fetch User table
cursor.execute("SELECT * FROM users LIMIT 5;")
rows = cursor.fetchall()
print(rows)



# Close the connection
conn.close()

# Return fetched context as a dictionary
# return {key: value for key, value in rows}

([1, 'bob@example.com', 'hashed_password'], [2, 'taylor@example.com', 'hashed_password'])
Tables in the database: ['topics', 'questions', 'titles', 'favorites', 'watchlater', 'activities', 'users', 'user_preferences', 'diet_types', 'recipes', 'tips', 'user_context']
Columns in 'users' table: ['uid', 'email', 'password']
